# Day 23 - Modulus problems or circular data structures?

* https://adventofcode.com/2020/day/23

I strongly suspect that part 2 will ask us to turn the knobs up on the number of rounds and number of cups, but for part 1 I just implemented the described steps (with a separate sorted list of labels to make it easier to determine the destination cup).

However, there may well be a mathematical approach to this problem, altough I don't immediately recognize the series yet.

In [1]:
from dataclasses import dataclass
from typing import List

@dataclass
class CupGame:
    cups: List[int]
    
    def __post_init__(self) -> None:
        self._labels = sorted(self.cups)
    
    @classmethod
    def from_string(cls, cups: str) -> "CupGame":
        return cls([int(c) for c in cups])

    def play_rounds(self, rounds: int = 100) -> str:
        cups, labels = self.cups, self._labels
        for _ in range(rounds):
            current = cups[0]
            removed = cups[1:4]
            idx = (labels.index(current) - 1) % len(labels)
            while labels[idx] in removed:
                idx = (idx - 1) % len(labels)
            destidx = cups.index(labels[idx]) + 1  # guaranteed to be < len(cups)
            cups[:] = [*cups[4:destidx], *removed, *cups[destidx:], cups[0]]
        # all labels after 1
        oneidx = cups.index(1)
        return "".join([str(c) for c in [*cups[oneidx + 1 :], *cups[:oneidx]]])

testcups = CupGame.from_string('389125467')
assert testcups.play_rounds() == '67384529'

In [2]:
import aocd
cups = CupGame.from_string(aocd.get_data(day=23, year=2020))

In [3]:
print("Part 1:", cups.play_rounds())

Part 1: 27865934
